In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
ticker = input("Enter the stock ticker symbol: ").strip().upper()
data = yf.download(ticker, period="1y") 
print(data.head()) 

Enter the stock ticker symbol:  aapl


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2024-03-25  170.047073  171.131947  168.653644  169.768390  54288300
2024-03-26  168.912430  170.614385  168.783036  169.201060  57388400
2024-03-27  172.495499  172.784144  169.310541  169.609134  60273300
2024-03-28  170.674103  171.420578  169.708660  170.942838  65672700
2024-04-01  169.230927  170.445194  168.683508  170.385479  46240500


In [3]:
def get_rsi(data, period=14):

    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss.replace(0, np.nan)  
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


In [4]:
def get_macd(data, short_window=12, long_window=26, signal_window=9):

    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

In [5]:
def get_bollinger_bands(data, window=20, num_std=2):
   
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bandwidth = (upper_band - lower_band) / rolling_mean
    return upper_band, lower_band, bandwidth


In [6]:
def get_momentum(data, period=14):
 
    return data['Close'].diff(period)


In [7]:
def get_volatility(data, period=14):
  
    return data['Close'].pct_change().rolling(window=period).std()


In [8]:
def get_stock_data(ticker):
    
    data = yf.download(ticker, period="1y") 
    return data

In [9]:
def prepare_data(data):
    
    
    data['Price_Change'] = data['Close'].pct_change()

    data['RSI'] = get_rsi(data)

    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    data['Momentum'] = get_momentum(data)

    data['Volatility'] = get_volatility(data)

    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False) 
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False)  

    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1  
    data.loc[sell_condition, 'Signal'] = -1 

    return data

In [10]:
def prepare_data(data):
    
    data['Price_Change'] = data['Close'].pct_change()

    data['RSI'] = get_rsi(data)

    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    data['Momentum'] = get_momentum(data)

    data['Volatility'] = get_volatility(data)

    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False)  
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False) 

    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1 
    data.loc[sell_condition, 'Signal'] = -1  

    data.dropna(inplace=True)
    return data

In [15]:
def train_model(data):
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    missing_cols = [col for col in features if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in data: {missing_cols}")

    X = data[features]
    y = data.get('Signal') 

    if y is None:
        raise ValueError("Target variable 'Signal' is missing from the dataset.")

    y = y.to_frame()

    imputer = SimpleImputer(strategy="mean")
    X = pd.DataFrame(imputer.fit_transform(X), columns=features, index=data.index)

    y.dropna(inplace=True)

    X = X.loc[y.index]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train.values.ravel())

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    best_rf = LogisticRegression()  # Example, replace with your actual model
    best_rf.fit(X_train, y_train)  # Ensure training is done
    with open('ML_Model.pkl', 'wb') as file:
       pickle.dump(best_rf, file)
    
    return model, accuracy 

In [16]:
def generate_alerts(data, model):
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    data['Predicted_Signal'] = model.predict(data[features])
    
    alerts = []
    if data['Predicted_Signal'].iloc[-1] == 1:
            alerts.append(("Buy", data.index[-1], data['Close'].iloc[-1]))
    elif data['Predicted_Signal'].iloc[-1] == -1:
            alerts.append(("Sell", data.index[-1], data['Close'].iloc[-1]))
    else:
            alerts.append(("Hold", data.index[-1], data['Close'].iloc[-1]))

    return alerts


In [18]:
def analyze_stock(ticker):
    
    data = get_stock_data(ticker)
    data = prepare_data(data)

    model, accuracy = train_model(data)  

    alerts = generate_alerts(data, model)

    print(f"\nAlerts for {ticker}:")
    for alert in alerts:
        print(f"{alert[0]} Signal at {alert[1]}: Price: {alert[2]}")

    return accuracy

if __name__ == "__main__":
   
    ticker = input("Enter the stock ticker symbol: ").strip().upper()

    accuracy = analyze_stock(ticker)  
    
    print(f"\nModel Accuracy for {ticker}: {accuracy:.2f}")

Enter the stock ticker symbol:  aapl


[*********************100%***********************]  1 of 1 completed


Alerts for AAPL:
Hold Signal at 2025-03-25 00:00:00: Price: Ticker
AAPL    222.175003
Name: 2025-03-25 00:00:00, dtype: float64

Model Accuracy for AAPL: 1.00



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
import joblib
import pickle